## Medical Device Recalls

### Step 1 - Collect Data 

Retriving enforcement report data from OpenFDA using API calls.

In [1]:
import json
import requests
import pandas as pd

In [2]:
BATCH_SIZE = 1000    # Maximum allowed by OpenFDA is 1000
FETCH_SIZE = 100000  # Enforcement report has around 29k records
DATE_RANGE = "[20200101+TO+20221231]"   # This is the accepted format to specify report date range
API_URL = "https://api.fda.gov/device/enforcement.json"

In [3]:
i = 0
contents = []

while True:
    print(f"Retriving batch #{i + 1}.", end=" ")
    skip = i * BATCH_SIZE
    resp = requests.get(f"{API_URL}?search=report_date:{DATE_RANGE}&limit={BATCH_SIZE}&skip={skip}")
    json_resp = json.loads(resp.text) 

    # OpenFDA imposes a limit on skip at 25000 meaning the maximum # of records retrived is 26000
    if "error" in json_resp:      
        print(f'Encountered an error: {json_resp["error"]["code"]}:{json_resp["error"]["message"]}.')
        print(f"DATA Request APT: {API_URL + str(skip)}")
        print("The remaining records are not retrived.")
        break

    contents += json_resp["results"]
    total = int(json_resp["meta"]["results"]["total"])

    i += 1
    print(f"Retrived {BATCH_SIZE * i}. Remaining {total - BATCH_SIZE * i}.")

    if i == 1:
        print(f"Total number of enforcement records: {total}.")

    if BATCH_SIZE * i >= FETCH_SIZE:
        break

    if BATCH_SIZE * i > total:
        break

df = pd.DataFrame(contents) 
df.shape

Retriving batch #1. Retrived 1000. Remaining 5904.
Total number of enforcement records: 6904.
Retriving batch #2. Retrived 2000. Remaining 4904.
Retriving batch #3. Retrived 3000. Remaining 3904.
Retriving batch #4. Retrived 4000. Remaining 2904.
Retriving batch #5. Retrived 5000. Remaining 1904.
Retriving batch #6. Retrived 6000. Remaining 904.
Retriving batch #7. Retrived 7000. Remaining -96.


(6904, 25)

In [4]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6904 entries, 0 to 6903
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   country                     6904 non-null   object
 1   city                        6904 non-null   object
 2   address_1                   6904 non-null   object
 3   reason_for_recall           6904 non-null   object
 4   address_2                   6904 non-null   object
 5   product_quantity            6904 non-null   object
 6   code_info                   6904 non-null   object
 7   center_classification_date  6904 non-null   object
 8   distribution_pattern        6904 non-null   object
 9   state                       6904 non-null   object
 10  product_description         6904 non-null   object
 11  report_date                 6904 non-null   object
 12  classification              6904 non-null   object
 13  openfda                     6904 non-null   obje

In [5]:
df.head()

,country,city,address_1,reason_for_recall,address_2,product_quantity,code_info,center_classification_date,distribution_pattern,state,...,recall_number,initial_firm_notification,product_type,event_id,recall_initiation_date,postal_code,voluntary_mandated,status,termination_date,more_code_info
0,United States,San Clemente,951 Calle Amanecer,Identification of a potential manufacturing ...,,Total of all products (Listed #1 thru 101) = 3...,Lot Number:4496182.,20201022,"Worldwide Distribution: US (nationwide): AL, A...",CA,...,Z-0272-2021,Letter,Devices,86054,20200706,92673-6212,Voluntary: Firm initiated,Ongoing,NaN,NaN
1,United States,Bloomington,750 N Daniels Way,The wire guide may be incorrectly loaded into ...,,13388 total,7953153 8077981 8070702 NS8176831 7999744 ...,20200204,Domestic distribution to nationwide US. For...,IN,...,Z-0977-2020,Letter,Devices,82026,20190104,47404-9120,Voluntary: Firm initiated,Terminated,20200529,NaN
2,United States,Waukesha,3000 N Grandview Blvd,The Event Notification Manager (ENM) functiona...,,47 installations,"Model: 5831882-012, Software Version 7.0 SP0.0...",20220725,Worldwide distribution - US Nationwide distrib...,WI,...,Z-1466-2022,Letter,Devices,90525,20220614,53188-1615,Voluntary: Firm initiated,Ongoing,NaN,
3,United States,Queensbury,603 Queensbury Ave,Specific serial numbers of the Solero MTA Gene...,,"26 US, 4 OUS",Serial Nos. QBY0002693 QBY0002376 QBY0002765 Q...,20210908,Domestic distribution Nationwide. Foreign dist...,NY,...,Z-2437-2021,,Devices,88444,20210720,12804-7619,Voluntary: Firm initiated,Ongoing,NaN,
4,United States,San Clemente,951 Calle Amanecer,Identification of a potential manufacturing ...,,Total of all products (Listed #1 thru 101) = 3...,Lot Number:4724881.,20201022,"Worldwide Distribution: US (nationwide): AL, A...",CA,...,Z-0218-2021,Letter,Devices,86054,20200706,92673-6212,Voluntary: Firm initiated,Ongoing,NaN,NaN


In [6]:
df.to_csv("../data/source/enforcement_reports.csv", index=False)

# The End.